### 코드 작성일 : 24.7.29 
### 코드 목표 : 대표 메뉴 변형

In [289]:
import os
from konlpy.tag import Komoran

if 'original_dir' not in globals() :
    original_dir = os.getcwd()
    original_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(original_dir)))))
    os.chdir(original_dir)
exec(open('setup/default.py').read())
#pd.set_option('display.max_rows', None) # 전체 컬럼 모두 출력


In [290]:
first_menu_list = pd.read_csv(original_dir + '/data/preproc/main/feature_lab_v1/unique_first_menu.csv')

In [291]:
# 단어 사전 만들어두기 
high_cat = pd.read_csv(original_dir+'/data/feature_lab/menu_map/entity/group/high_cat.csv').drop_duplicates()['high_cat'].tolist()
med_cat = pd.read_csv(original_dir+'/data/feature_lab/menu_map/entity/group/med_cat.csv').drop_duplicates()['med_cat'].tolist()
ingredient = pd.read_csv(original_dir+'/data/feature_lab/menu_map/entity/group/ingredient.csv').drop_duplicates()['ingredient'].tolist()
adj_cat = pd.read_csv(original_dir+'/data/feature_lab/menu_map/entity/group/adj_cat.csv').drop_duplicates()['adj_cat'].tolist()

custom_nouns = high_cat+med_cat+ingredient+adj_cat
with open(original_dir+'/data/feature_lab/menu_map/entity/group/user_dic.txt', 'w', encoding='utf-8') as f:
    for noun in custom_nouns:
        f.write(f"{noun}\tNNG\n")

konlp = Komoran(userdic=original_dir+'/data/feature_lab/menu_map/entity/group/user_dic.txt')

In [292]:
def konlp_split (text) :
    res = konlp.nouns(text)
    if len(res)==0 : 
        res = konlp.morphs(text)
    return res

first_menu_list['menu_comp'] = first_menu_list['menu'].progress_apply(konlp_split)

100%|██████████| 648/648 [00:00<00:00, 12595.04it/s]


In [293]:
synonyms_df = pd.read_csv(original_dir + '/data/feature_lab/menu_map/entity/synonym.csv')
synonyms = dict(zip(synonyms_df['same'], synonyms_df['top']))

# 메뉴 이름을 교체하는 함수
def replace_with_synonym(menu_items):
    # 동의어 사전에 있으면 교체, 없으면 원래 값 유지
    return [synonyms.get(item, item) for item in menu_items]

first_menu_list['menu_comp'] = first_menu_list['menu_comp'].progress_apply(replace_with_synonym)

100%|██████████| 648/648 [00:00<00:00, 1060854.41it/s]


In [294]:
import copy
# 'etc' 컬럼에 menu_comp의 깊은 복사본 할당
first_menu_list['etc'] = first_menu_list['menu_comp'].apply(copy.deepcopy)

def find_and_remove_cat(etc_list, cat_values):
    for item in reversed(etc_list):
        if item in cat_values:
            etc_list.remove(item)  # 원 리스트에서 삭제 
            return item
    return None

def find_and_remove_all_cat(etc_list, cat_values):
    matched_items = [item for item in etc_list if item in cat_values]
    etc_list[:] = [item for item in etc_list if item not in cat_values]
    return list(set(matched_items)) if matched_items else None


first_menu_list['high_cat'] = first_menu_list['etc'].apply(lambda x: find_and_remove_cat(x, high_cat) if x else None)
first_menu_list['med_cat'] = first_menu_list['etc'].apply(lambda x: find_and_remove_cat(x, med_cat) if x else None)
first_menu_list['ingredient'] = first_menu_list['etc'].apply(lambda x: find_and_remove_all_cat(x, ingredient+high_cat) if x else None)


In [295]:
condition = first_menu_list['high_cat'].isna() & first_menu_list['med_cat'].isna() & first_menu_list['ingredient'].isna()
first_menu_list.loc[condition, 'ingredient'] = first_menu_list.loc[condition, 'menu'].apply(lambda x: [x])


In [296]:
first_menu_list.to_csv(original_dir + '/data/preproc/main/feature_lab_v1/unique_first_menu_w_component.csv',index=False)

In [297]:
high_cat = pd.read_csv(original_dir+'/data/feature_lab/menu_map/entity/group/high_cat.csv')
def calculate_sim_high_cat(x, target_high_cat):
    # target_high_cat의 big_category 추출
    target_category = high_cat.loc[high_cat.high_cat == target_high_cat, 'big_category']
    # x의 big_category 추출
    x_category = high_cat.loc[high_cat.high_cat == x, 'big_category']
    
    # target_category와 x_category가 비어있지 않은지 확인
    target_cat_value = target_category.values[0] if not target_category.empty else None
    x_cat_value = x_category.values[0] if not x_category.empty else None

    # 조건에 따라 값을 반환
    if target_cat_value == '고기' and x is None:
        return 0.5
    if x_cat_value == '고기' and target_high_cat is None:
        return 0.5
    
    if target_high_cat is None and x is None: 
        return None
    if x == target_high_cat:  # 완전 동일
        return 1
    elif target_cat_value == x_cat_value:  # 유사 식품
        return 0.7
    else:
        return 0


def calculate_sim_ingredient(x, target_ingredient):
    x_set = set(x or [])
    target_set = set(target_ingredient or [])
    intersection_len = len(target_set & x_set)
    min_len = min(len(target_set), len(x_set))
    return intersection_len / min_len if min_len > 0 else 0


def calculate_sim(x, y, z):
    if x is None : 
        return z 
    else :
        return x*0.4 + y*0.2 + z*0.4

In [298]:
fin_sim_df.iloc[:5]

,target_menu,menu,menu_comp,etc,high_cat,med_cat,ingredient,high_cat_sim,med_cat_sim,ingredient_sim,sim_score
0,투움바파스타,투움바파스타,"[투움바, 파스타]",[],파스타,None,[투움바],1.0,1,1.0,1.0
1,투움바파스타,닭곰탕,"[닭, 곰, 탕]",[],탕,곰,[닭],0.0,0,0.0,0.0
2,투움바파스타,육감쫄면,"[육감, 쫄, 면]",[육감],면,쫄,None,0.0,0,0.0,0.0
3,투움바파스타,차돌된장찌개,"[차돌, 된장, 찌개]",[],찌개,None,"[차돌, 된장]",0.0,1,0.0,0.2
4,투움바파스타,매실청돈육불고기,"[매실청, 돼지, 불고기]",[],None,None,"[돼지, 불고기, 매실청]",0.0,1,0.0,0.2


In [299]:
fin_sim_df = pd.DataFrame()
for i in tqdm(range(first_menu_list.shape[0])):
    target_menu = first_menu_list.menu[i]
    target_high_cat = first_menu_list.high_cat[i]
    target_med_cat = first_menu_list.med_cat[i]
    target_ingredient = first_menu_list.ingredient[i]

    relate_df = first_menu_list.copy()
    relate_df['high_cat_sim'] = relate_df['high_cat'].apply(lambda x: calculate_sim_high_cat(x,target_high_cat))
    relate_df['med_cat_sim'] = relate_df['med_cat'].apply(lambda x: 1 if x==target_med_cat else 0)
    relate_df['ingredient_sim'] = relate_df['ingredient'].apply(lambda x: calculate_sim_ingredient(x,target_ingredient))
    relate_df['sim_score'] = relate_df.apply(
    lambda row: calculate_sim(row['high_cat_sim'], row['med_cat_sim'], row['ingredient_sim']),
    axis=1
    )   
    
    relate_df.insert(0, 'target_menu', target_menu)
    fin_sim_df = pd.concat([fin_sim_df, relate_df])

100%|██████████| 648/648 [01:00<00:00, 10.74it/s]


In [300]:
fin_sim_df.sort_values(by=['target_menu','sim_score'],ascending=False).\
to_csv(original_dir + '/data/preproc/main/feature_lab_v1/first_menu_similarity.csv',index=False)